In [21]:
import sys
import numpy as np
import math
import pandas as pd

In [48]:
amoc_one = pd.read_csv('amoc_4traj_100y_block1.csv')

In [50]:
amoc_one.head()
amoc_one['S0,T0'].unique()

array(['0.5,0.5', '0.5,1', '1,0.5', '1,1'], dtype=object)

In [38]:
#split into trajectories for input
#create a unique dataframe for each unique value in S0,T0 column
unique_starts = amoc_one['S0,T0'].unique()
traj_dfs = []
traj_list = []
last_amocs = []
last_temps = []
last_salts = []
for start in range(len(unique_starts)):
    trajectory = amoc_one.loc[amoc_one['S0,T0']==unique_starts[start]]
    amoc_list = trajectory['AMOC'].tolist()
    traj_dfs.append(trajectory)
    traj_list.append(amoc_list)

for traj in traj_dfs:
    #print(traj)
    last_amoc = traj['AMOC'].values[-1]
    last_temp = traj['T'].values[-1]
    last_salt = traj['S'].values[-1]
    last_amocs.append(last_amoc)
    last_temps.append(last_temp)
    last_salts.append(last_salt)

[0.0886592680069116, 0.1676374046234617, -0.0062382948538322, -0.0234503009368366]


In [39]:
# input parameters
#number of trajectories
ntrajs=4
#number of timepoints per block
ntblock=100
#delta t
dt=0.01
#counter index, shows which block we are on now
block=1
#very important tuning parameter for moving distribution of new function, high k biases towards high values of AMOC and vice versa
k=-3

In [41]:
# resampling

# compute weights
obsint=np.empty(ntrajs)
for traj in range(ntrajs):
    obsint[traj]=np.sum(traj_list[traj])*dt
W=np.exp(k*(obsint))
print(W)
# compute the normalizer and the normalized weights
R=sum(W)/ntrajs
w=W/R

# compute the number of clones generated by trajectory
nc=np.empty(ntrajs)
for traj in range(ntrajs):
    nc[traj]=math.floor(w[traj]+np.random.uniform(0,1))

# compute total number of clones and the difference with original ensemble size 
Nc=sum(nc)
dN=Nc-ntrajs

[0.79678127 0.52095251 1.29117586 1.19748563]


In [42]:
'''
if the clones are less than what is needed, clone randomly among the successfull 
trajectories, proportionally to the number of clones they are originally meant 
to produce, with repetition
'''
label=np.arange(ntrajs)
print(label)
if dN<0:
    traj2clone=np.empty(int(-dN))
    for traj in range(int(-dN)):    
        traj2clone[traj]=np.random.choice(label,p=nc/sum(nc))  
    for traj in range(int(-dN)):
        nc[int(traj2clone[traj])]=nc[int(traj2clone[traj])]+1
        
'''
if the clones are more than what is needed, kill randomly among the successfull 
trajectories, proportionally to the number of clones they are originally meant 
to produce, without repetition
'''

if dN>0:
    traj2kill=np.empty(int(dN))
    for traj in range(int(dN)):
        traj2kill[traj]=np.random.choice(label,p=nc/sum(nc))
        nc[int(traj2kill[traj])]=nc[int(traj2kill[traj])]-1

'''
note on the procedure above: the random selection is performed proportionally 
to the number of clones the trajectories are originally meant to produce. This 
means selecting on the population of trajectories present after the resampling: 
what is done above is a correction to the resampling.

define a label array with 0 if the trajectory survives, 1 if it is killed and 
it is meant to be substituted by a copy of a successfull trajectory
'''
killedlabel=np.empty(ntrajs)
for traj in range(ntrajs):
    if nc[traj]>0:
        killedlabel[traj]=0
    else:
        killedlabel[traj]=1


[0 1 2 3]


In [43]:
'''
for each trajectory, determine the label of the trajectory whose restart will 
be used to continue the evolution. Originally this initialisation label is equal 
to the label of the trajectory itself: the trajectory continues its own evolution.
If nc is 0 (the trajectory will be killed) or 1 (the trajectory survives but does not  
generate clones) nothing is done. If nc>1 (the trajectory spawns nc-1 clones, 
in addition to continuing its own evolution), pick randomly nc-1 trajectories 
among those meant to be killed, and give them the label of the spawning trajectory.
The label array of the trajectories to be killed is updated every time, in order to
be sure to substitute all the killed trajectories by the end of the loop.  

'''
initlabel=np.arange(ntrajs)
udkilledlabel=np.empty(ntrajs)
udkilledlabel[:]=killedlabel[:]
for traj in range(ntrajs):
    if nc[traj]>1:
        for n in range(int(nc[traj])-1):
            traj2substitute=np.random.choice(label,p=udkilledlabel/sum(udkilledlabel))
            initlabel[int(traj2substitute)]=traj
            udkilledlabel[int(traj2substitute)]=0

print(initlabel)
print(udkilledlabel)

[2 1 2 3]
[0. 0. 0. 0.]


In [51]:
#df for metadata
metadata = pd.DataFrame(data={'Initial S,T': S0_T0,'Last AMOC value':last_amocs,'Corresponding T value': last_temps,
                              'Corresponding S Value': last_salts, 'Label':label,'Exponential':W,'Normalization,R':R,
                              'Weight':w,'Number of Clones':nc,'Killed?':killedlabel,'Init Label':initlabel})

In [53]:
metadata.head(50)

,"Initial S,T",Last AMOC value,Corresponding T value,Corresponding S Value,Label,Exponential,"Normalization,R",Weight,Number of Clones,Killed?,Init Label
0,"0.5,0.5",0.088659,0.636767,0.548108,0,0.796781,0.951599,0.837308,0.0,1.0,2
1,"0.5,1",0.167637,0.551521,0.383884,1,0.520953,0.951599,0.547450,1.0,0.0,1
2,"1,0.5",-0.006238,0.729938,0.736176,2,1.291176,0.951599,1.356849,2.0,0.0,2
3,"1,1",-0.023450,0.708566,0.732016,3,1.197486,0.951599,1.258393,1.0,0.0,3


In [55]:
# from here on change how to save output depending on the model
#add perturbation by determining if the traj has been killed, if it has add the perturbation to the temp column

'''
if killed is 1, 
New T/S value depends on init label, equal to value of T/S where killed is 0 and init label is equal
add perturbation 1 +Er to temperature
Add column for new temp values

Next steps would be: redo k, redo this again
Then try to optimize it, and try to run through whole simulation
'''

#get killed trajectories to add perturbation to
killed=metadata['Killed?'].tolist()
init_temps = []
init_salts = []
for kill in range(len(killed)):
    if (killed[kill]==1.0):
        #change the last temps and salinities to clone
        init_label = metadata['Init Label'].values[kill]
        #print("Init label: ",init_label)
        mother_filter = metadata[(metadata['Init Label'] == init_label) & (metadata['Killed?'] == 0.0)]
        mother_temp = mother_filter['Corresponding T value'].iloc[0]
        mother_salt = mother_filter['Corresponding S Value'].iloc[0]
        #print(mother_temp,mother_salt)
        #define noise, amplitude
        epsilon = 10e-6
        r=np.random.uniform(-1,1)
        init_temps.append(mother_temp*(1+epsilon*r))
        #print("mother Temp: ", mother_temp)
        #print("mother Salt: ",mother_salt)
        init_salts.append(mother_salt)
    else:
        init_temps.append(last_temps[kill])
        init_salts.append(last_salts[kill])


metadata['Initial Temperatures Next Block'] = init_temps
metadata['Initial Salinities Next Block'] = init_salts
        
metadata.head()

,"Initial S,T",Last AMOC value,Corresponding T value,Corresponding S Value,Label,Exponential,"Normalization,R",Weight,Number of Clones,Killed?,Init Label,Initial Temperatures Next Block,Initial Salinities Next Block
0,"0.5,0.5",0.088659,0.636767,0.548108,0,0.796781,0.951599,0.837308,0.0,1.0,2,0.729938,0.736176
1,"0.5,1",0.167637,0.551521,0.383884,1,0.520953,0.951599,0.547450,1.0,0.0,1,0.551521,0.383884
2,"1,0.5",-0.006238,0.729938,0.736176,2,1.291176,0.951599,1.356849,2.0,0.0,2,0.729938,0.736176
3,"1,1",-0.023450,0.708566,0.732016,3,1.197486,0.951599,1.258393,1.0,0.0,3,0.708566,0.732016


In [31]:
metadata.to_csv('metadata_initial.csv',index=False)